# Introduction to Probability and Statistics
In this notebook, we will play around with some of the concepts we have previously discussed. Many concepts from probability and statistics are well-represented in major libraries for data processing in Python, such as `numpy` and `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## 隨機變量與分佈
讓我們從介乎 0 至 9 的均勻分佈中抽取 30 個樣本值。 我們亦會計算平均數和變異數。


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

為了視覺上估計樣本中有多少不同的值，我們可以繪製**直方圖**：


In [ ]:
plt.hist(sample)
plt.show()

## 分析實際數據

均值和方差在分析實際世界數據時非常重要。讓我們從[SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)載入有關棒球運動員的數據。


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> 我們在這裏使用一個名為 [**Pandas**](https://pandas.pydata.org/) 的套件來進行數據分析。我們會在這門課程稍後討論更多關於 Pandas 和在 Python 中處理數據的內容。

讓我們計算年齡、身高和體重的平均值：


In [ ]:
df[['Age','Height','Weight']].mean()

現在讓我們專注於身高，並計算標準差和變異數：


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

除了平均值之外，查看中位數和四分位數也是很有意義的。它們可以用**箱型圖**來視覺化：


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

我哋亦可以對數據集中嘅子集製作箱形圖，例如按球員角色分組。


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **注意**：這個圖表表明，平均而言，一壘手的身高較二壘手的身高高。稍後我們會學習如何更正式地檢驗這個假設，以及如何證明我們的數據在統計上顯著支持這一點。  

年齡、身高和體重都是連續隨機變數。你認為它們的分佈是什麼？一個好方法是繪製值的直方圖：  


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 常態分佈

讓我們創建一個人工重量樣本，該樣本遵循與我們的真實數據具有相同均值和方差的常態分佈：


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

由於現實生活中大多數數值呈常態分佈，因此我們不應使用均勻隨機數生成器來生成樣本數據。以下是如果我們嘗試使用均勻分佈（由 `np.random.rand` 生成）來生成體重數據時會發生的情況：


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## 置信區間

現在讓我們計算棒球運動員體重及身高的置信區間。我們將使用[這個stackoverflow討論中的程式碼](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data)：


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## 假設檢定

讓我們探索我們棒球選手資料集中的不同角色：


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

讓我們測試一個假設，即一壘手的身高比二壘手高。最簡單的方法是檢驗置信區間：


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

我哋可以見到啲區間冇重疊。

一個統計上更正確嘅方法去證明呢個假設係用**Student t檢定**：


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

`ttest_ind` 函式返回的兩個值是：
* p 值可以被視為兩個分佈具有相同平均值的機率。在我們的案例中，該值非常低，表示有強烈證據支持一壘手較高。
* t 值是標準化平均差的中間值，用於 t 檢定，並且會與給定信心水準的閾值進行比較。


## 模擬常態分佈與中心極限定理

Python 的偽隨機生成器設計成給我們一個均勻分佈。如果我們想要建立一個常態分佈的生成器，可以使用中心極限定理。要取得常態分佈的值，我們只需計算均勻生成樣本的平均值。


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## 相關性與邪惡棒球公司

相關性讓我們能夠找到數據序列之間的關係。在我們的玩具範例中，假設有一家邪惡的棒球公司根據球員的身高支付薪水——球員越高，薪水越多。假設有一個基本薪資為$1000，並根據身高額外獎勵$0到$100。我們將採用MLB的真實球員，並計算他們的虛擬薪水：


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

現在讓我們計算這些序列的協方差和相關係數。`np.cov` 將會給我們所謂的**協方差矩陣**，這是協方差對多個變量的擴展。協方差矩陣 $M$ 的元素 $M_{ij}$ 是輸入變量 $X_i$ 和 $X_j$ 之間的協方差，而對角線上的值 $M_{ii}$ 是 $X_{i}$ 的變異數。類似地，`np.corrcoef` 將會給我們**相關矩陣**。


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

相關係數等於1表示兩個變數之間存在強烈的**線性關係**。我們可以通過繪製一個值與另一個值的圖形來直觀地看到這種線性關係：


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

讓我們看看如果關係不是線性會發生什麼。假設我們的公司決定隱藏身高和薪水之間明顯的線性依賴，並在公式中引入一些非線性，如 `sin`：


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

在這種情況下，相關性稍微小一些，但仍然相當高。現在，為了使關係更不明顯，我們可能想通過向薪水中加入一些隨機變量來增加額外的隨機性。讓我們看看會發生什麼：


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> 你能猜到為什麼點會排成這樣的垂直線嗎？

我們已經觀察到像薪水這種人工設計的概念與觀察變量*身高*之間的相關性。現在讓我們也看看兩個觀察變量，比如身高和體重，是否也存在相關性：


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

不幸的是，我們沒有得到任何結果——只有一些奇怪的 `nan` 值。這是因為我們序列中的一些值未定義，表示為 `nan`，這導致運算結果也未定義。從矩陣中我們可以看到，`Weight` 是有問題的欄位，因為已經計算了 `Height` 值之間的自我相關。

> 此範例顯示了**資料準備**和**清理**的重要性。沒有適當的資料，我們無法計算任何東西。

讓我們使用 `fillna` 方法來填補缺失值，並計算相關性：


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

確實存在相關性，但沒有我們人工示例中的那麼強烈。事實上，如果我們觀察一個數值與另一個數值的散點圖，這種關係會不那麼明顯：


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusion

在本筆記本中，我們學會了如何對資料進行基本操作以計算統計函數。我們現在知道如何使用完善的數學和統計工具來驗證某些假設，以及如何根據資料樣本計算任意變數的信賴區間。


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件是使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們努力確保準確性，但請注意自動翻譯可能包含錯誤或不準確之處。請以原文文件為權威資料來源。對於重要資訊，建議採用專業人工翻譯。對於因使用此翻譯而引起的任何誤解或錯誤詮釋，我們概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
